In [2]:
import osmapi, time, overpy

In [3]:
username = 'schleuss' # you should make this yours

In [6]:
# open condos file
f = open("condos.csv","r").read()
ains = f.split(',')

# start not found condo file
j = open("condos-left.csv","a")

# crack open APIs
api = overpy.Overpass()
api2 = osmapi.OsmApi(passwordfile = "/Users/jschleuss/Documents/osm.password") # reset this path to yours

2010023059


In [16]:
# find id by ain
def findBuildings(ain):    
    result = api.query("""[out:json][timeout:25];(way["lacounty:ain"="""+ain+"""];relation["lacounty:ain"="""+ain+"""];);out body;>;out skel qt;""")
    
    # safe spot to store ids
    ways = []
    relations = []

    # loop through both ways and relations
    for way in result.ways:
        if "building" in way.tags:
            ways.append(way.id)
            print way.tags
    
    for relation in result.relations:
        if "building" in relation.tags:
            relations.append(relation.id)
            print relation.tags

    print ways
    print relations
    
    # must be nice
    time.sleep(1)

In [15]:
# testing to find buildings
# selain = f[2]
findBuildings('7563006104')

for ain in ains[:10]:
    findBuildings(ain)

{u'building': u'apartments', u'lacounty:bld_id': u'462490723139', u'lacounty:ain': u'7563006104', u'height': u'2.2', u'ele': u'124.1', u'start_date': u'1969'}
{u'building': u'apartments', u'lacounty:bld_id': u'462732723080', u'lacounty:ain': u'7563006104', u'height': u'8.3', u'ele': u'129.0', u'type': u'multipolygon', u'start_date': u'1969'}
[407339465]
[6100288]
[]
[]
{u'building': u'residential', u'lacounty:bld_id': u'468053736360', u'lacounty:ain': u'7442032032', u'height': u'8.5', u'ele': u'50.1', u'start_date': u'1980'}
[407729012]
[]
{u'building': u'apartments', u'lacounty:bld_id': u'463670722850', u'lacounty:ain': u'7563006036', u'height': u'5.2', u'ele': u'121.2', u'start_date': u'1984'}
{u'building': u'apartments', u'lacounty:bld_id': u'463616722707', u'lacounty:ain': u'7563006036', u'height': u'9.7', u'ele': u'123.1', u'start_date': u'1984'}
{u'building': u'apartments', u'lacounty:bld_id': u'463508722914', u'lacounty:ain': u'7563006036', u'height': u'7.9', u'ele': u'125.2', u

In [9]:
def updateBuildings(ain):    
    result = api.query("""[out:json][timeout:25];(way["lacounty:ain"="""+ain+"""];relation["lacounty:ain"="""+ain+"""];);out body;>;out skel qt;""")
            
    # safe spot to store ids
    ways = []
    relations = []
    
    # loop through both ways and relations
    for way in result.ways:
        if "building" in way.tags:
            ways.append(way.id)
            
    for relation in result.relations:
        if "building" in relation.tags:
            relations.append(relation.id)

    if len(ways) > 0 or len(relations) > 0:
        print ain + " has " str(len(ways)+len(relations)) + " features"
        
        # open a changeset
        api2.ChangesetCreate({u"comment": u"Removing building:units from condo and adding apartments type", u'source':'L.A. County Assessor', u'created_by':username})    

        # loop through the ways
        for way in ways:
            obj = api2.WayGet(way) # get object

            # delete unit count and change building to residential 
            if 'building:units' in obj['tag']:
                print obj['tag']['lacounty:ain'] + " removing " + obj['tag']['building:units'] + " units from way:" + str(way)
                del obj['tag']['building:units']

            if obj['tag']['building'] != 'apartments':
                print obj['tag']['lacounty:ain'] + " updating " + obj['tag']['building'] + " to apartments way:" + str(way)
                obj['tag']['building'] = 'apartments'

            obj = api2.WayUpdate(obj) # update the object

            # wait a beat
            time.sleep(1)

        # loop through the relations
        for relation in relations:
            obj = api2.RelationGet(relation) # get object

            # delete unit count and change building to residential 
            if 'building:units' in obj['tag']:
                print obj['tag']['lacounty:ain'] + " removing " + obj['tag']['building:units'] + " units from relation:" + str(relation)
                del obj['tag']['building:units']

            if obj['tag']['building'] != 'apartments':
                print obj['tag']['lacounty:ain'] + " updating " + obj['tag']['building'] + " to apartments relation" + str(relation)
                obj['tag']['building'] = 'apartments'

            obj = api2.RelationUpdate(obj) # update the object

            # wait a beat
            time.sleep(1)            
            
        api2.ChangesetClose() # bye changeset!
        
    else:
        j.write(ain + '\n') # add failing ain
        print "nothing found for " + ain
        
    # wait a beat
    time.sleep(1)

In [10]:
# api2.ChangesetClose()
# testing to update buildings
# api2.ChangesetClose()
updateBuildings('7465001034')

for ain in ains[:10]:
    updateBuildings(ain)
    time.sleep(1)

''
''
2010023047 removing 1 units from way:431849500
2010023047 updating house to apartments way:431849500
2010023047 removing 1 units from way:431849560
2010023047 updating house to apartments way:431849560
2010023047 removing 1 units from way:431849610
2010023047 updating house to apartments way:431849610
2010023047 removing 1 units from way:431849627
2010023047 updating house to apartments way:431849627
2010023047 removing 1 units from way:431849773
2010023047 updating house to apartments way:431849773
2010023047 removing 1 units from way:431849888
2010023047 updating house to apartments way:431849888
2010023047 removing 1 units from way:431849900
2010023047 updating house to apartments way:431849900
2010023047 removing 1 units from way:431850163
2010023047 updating house to apartments way:431850163
2010023047 removing 1 units from way:431850198
2010023047 updating house to apartments way:431850198
2010023047 removing 1 units from way:431850225
2010023047 updating house to apartment

In [25]:
j.close()